# Notebook 08: Electronic Properties

## Calculate ONLY After Structure is Validated and Stable!

### Prerequisites Reminder

Before calculating electronic properties, ensure:

- ☑ Structure from reliable database
- ☑ Convergence testing complete
- ☑ Structure optimized
- ☑ **Stability verified** (thermodynamic, dynamic, mechanical)

**If ANY step is skipped → Results are MEANINGLESS!**

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import re
from typing import Dict, List, Tuple, Optional

# Constants
RY_TO_EV = 13.605693122994
HBAR = 1.054571817e-34  # J·s
ME = 9.1093837015e-31   # kg

---

## 1. Band Structure Calculation Workflow

```
Step 1: SCF calculation (dense k-mesh)
        ↓
Step 2: Bands calculation (k-path through BZ)
        ↓
Step 3: Post-processing with bands.x
        ↓
Step 4: Plot and analyze
```

In [ ]:
# High-symmetry points for different crystal systems
HIGH_SYMMETRY_POINTS = {
    'FCC': {
        'G': (0.000, 0.000, 0.000),  # Gamma
        'X': (0.500, 0.000, 0.500),
        'W': (0.500, 0.250, 0.750),
        'K': (0.375, 0.375, 0.750),
        'L': (0.500, 0.500, 0.500),
        'U': (0.625, 0.250, 0.625),
    },
    'BCC': {
        'G': (0.000, 0.000, 0.000),
        'H': (0.500, -0.500, 0.500),
        'N': (0.000, 0.000, 0.500),
        'P': (0.250, 0.250, 0.250),
    },
    'HEX': {
        'G': (0.000, 0.000, 0.000),
        'M': (0.500, 0.000, 0.000),
        'K': (0.333, 0.333, 0.000),
        'A': (0.000, 0.000, 0.500),
        'L': (0.500, 0.000, 0.500),
        'H': (0.333, 0.333, 0.500),
    },
    'CUBIC': {
        'G': (0.000, 0.000, 0.000),
        'X': (0.500, 0.000, 0.000),
        'M': (0.500, 0.500, 0.000),
        'R': (0.500, 0.500, 0.500),
    }
}

# Standard paths
STANDARD_PATHS = {
    'FCC': [('G', 20), ('X', 10), ('W', 10), ('K', 20), ('G', 20), ('L', 0)],
    'BCC': [('G', 20), ('H', 20), ('N', 20), ('G', 20), ('P', 0)],
    'HEX': [('G', 20), ('M', 20), ('K', 20), ('G', 20), ('A', 0)],
    'CUBIC': [('G', 20), ('X', 20), ('M', 20), ('G', 20), ('R', 0)],
}

def generate_kpath_card(crystal_system: str, npoints: int = 20) -> str:
    """
    Generate K_POINTS {crystal_b} card for band structure calculation.
    
    Parameters
    ----------
    crystal_system : str
        'FCC', 'BCC', 'HEX', or 'CUBIC'
    npoints : int
        Points between high-symmetry points
    
    Returns
    -------
    str : K_POINTS card for QE input
    """
    if crystal_system not in HIGH_SYMMETRY_POINTS:
        print(f"Unknown crystal system: {crystal_system}")
        return None
    
    points = HIGH_SYMMETRY_POINTS[crystal_system]
    path = STANDARD_PATHS[crystal_system]
    
    lines = ["K_POINTS {crystal_b}"]
    lines.append(str(len(path)))
    
    for name, npts in path:
        coords = points[name]
        lines.append(f"  {coords[0]:.6f} {coords[1]:.6f} {coords[2]:.6f} {npts}  ! {name}")
    
    return '\n'.join(lines)

# Example
print("K-path for FCC (Silicon):")
print("=" * 50)
print(generate_kpath_card('FCC'))

---

## 2. Bands Input File Generator

In [ ]:
def generate_bands_input(prefix: str, ecutwfc: float, ecutrho: float,
                         pseudo_dir: str, crystal_system: str,
                         cell_parameters, atomic_species, atomic_positions,
                         nbnd: int = None) -> str:
    """
    Generate bands calculation input file.
    
    This assumes SCF has already been run with the same prefix.
    """
    nat = len(atomic_positions)
    ntyp = len(atomic_species)
    
    kpath = generate_kpath_card(crystal_system)
    
    input_lines = f"""&CONTROL
    calculation = 'bands'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
/

&SYSTEM
    ibrav = 0
    nat = {nat}
    ntyp = {ntyp}
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
"""
    
    if nbnd:
        input_lines += f"    nbnd = {nbnd}\n"
    
    input_lines += """/

&ELECTRONS
    conv_thr = 1.0e-8
/

ATOMIC_SPECIES
"""
    
    for symbol, mass, pp_file in atomic_species:
        input_lines += f"    {symbol}  {mass}  {pp_file}\n"
    
    input_lines += "\nCELL_PARAMETERS {angstrom}\n"
    for vec in cell_parameters:
        input_lines += f"    {vec[0]:12.8f}  {vec[1]:12.8f}  {vec[2]:12.8f}\n"
    
    input_lines += "\nATOMIC_POSITIONS {crystal}\n"
    for symbol, x, y, z in atomic_positions:
        input_lines += f"    {symbol}  {x:12.8f}  {y:12.8f}  {z:12.8f}\n"
    
    input_lines += "\n" + kpath
    
    return input_lines

# Example for Silicon
print("\nBands Input for Silicon:")
print("=" * 50)
print("(Showing structure of generated input)\n")

---

## 3. Parsing Band Structure Output

In [ ]:
def parse_bands_gnu(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Parse bands.dat.gnu file from QE bands.x.
    
    The GNU format has:
    - k-distance and energy for each band
    - Blank lines separate bands
    
    Returns
    -------
    k_distances : np.ndarray
        K-point distances along path
    bands : np.ndarray
        Band energies, shape (nkpts, nbands)
    """
    data = []
    current_band = []
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:  # Empty line = new band
                if current_band:
                    data.append(current_band)
                    current_band = []
            else:
                parts = line.split()
                if len(parts) >= 2:
                    k = float(parts[0])
                    e = float(parts[1])
                    current_band.append((k, e))
    
    if current_band:
        data.append(current_band)
    
    if not data:
        return None, None
    
    k_distances = np.array([p[0] for p in data[0]])
    bands = np.array([[p[1] for p in band] for band in data]).T
    
    return k_distances, bands

def plot_band_structure(k_distances: np.ndarray, bands: np.ndarray,
                        fermi_energy: float = 0.0,
                        high_sym_positions: List[float] = None,
                        high_sym_labels: List[str] = None,
                        title: str = 'Band Structure',
                        save_path: str = None):
    """
    Plot band structure with proper formatting.
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Shift energies relative to Fermi level
    bands_shifted = bands - fermi_energy
    
    # Plot bands
    for i in range(bands.shape[1]):
        ax.plot(k_distances, bands_shifted[:, i], 'b-', linewidth=1.5)
    
    # Add Fermi level
    ax.axhline(y=0, color='r', linestyle='--', linewidth=1, label='$E_F$')
    
    # High-symmetry lines
    if high_sym_positions:
        for pos in high_sym_positions:
            ax.axvline(x=pos, color='gray', linestyle='--', alpha=0.5)
    
    # Labels
    if high_sym_positions and high_sym_labels:
        ax.set_xticks(high_sym_positions)
        ax.set_xticklabels(high_sym_labels, fontsize=14)
    
    ax.set_xlabel('Wave Vector', fontsize=14)
    ax.set_ylabel('Energy (eV)', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.set_xlim(k_distances.min(), k_distances.max())
    ax.grid(True, alpha=0.3, axis='y')
    ax.legend()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()
    
    return fig, ax

print("Band structure plotting functions defined.")

---

## 4. Density of States (DOS)

### DOS Workflow

```
Step 1: SCF calculation (same as before)
        ↓
Step 2: NSCF with VERY dense k-mesh
        ↓
Step 3: dos.x for total DOS
        ↓
Step 4: projwfc.x for projected DOS (PDOS)
```

In [ ]:
def generate_nscf_input(prefix: str, ecutwfc: float, ecutrho: float,
                        pseudo_dir: str, kpoints: Tuple[int, int, int],
                        cell_parameters, atomic_species, atomic_positions) -> str:
    """
    Generate NSCF input for DOS calculation.
    
    Note: K-mesh should be MUCH denser than SCF (e.g., 16×16×16 or more)
    """
    nat = len(atomic_positions)
    ntyp = len(atomic_species)
    kx, ky, kz = kpoints
    
    input_text = f"""&CONTROL
    calculation = 'nscf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
/

&SYSTEM
    ibrav = 0
    nat = {nat}
    ntyp = {ntyp}
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'tetrahedra'
/

&ELECTRONS
    conv_thr = 1.0e-8
/

ATOMIC_SPECIES
"""
    
    for symbol, mass, pp_file in atomic_species:
        input_text += f"    {symbol}  {mass}  {pp_file}\n"
    
    input_text += "\nCELL_PARAMETERS {angstrom}\n"
    for vec in cell_parameters:
        input_text += f"    {vec[0]:12.8f}  {vec[1]:12.8f}  {vec[2]:12.8f}\n"
    
    input_text += "\nATOMIC_POSITIONS {crystal}\n"
    for symbol, x, y, z in atomic_positions:
        input_text += f"    {symbol}  {x:12.8f}  {y:12.8f}  {z:12.8f}\n"
    
    input_text += f"\nK_POINTS {{automatic}}\n    {kx} {ky} {kz} 0 0 0\n"
    
    return input_text

def generate_dos_input(prefix: str, emin: float = -15.0, emax: float = 15.0,
                       deltae: float = 0.01, fildos: str = 'dos.dat') -> str:
    """
    Generate dos.x input file.
    """
    return f"""&DOS
    prefix = '{prefix}'
    outdir = './tmp'
    fildos = '{fildos}'
    Emin = {emin}
    Emax = {emax}
    DeltaE = {deltae}
/
"""

def parse_dos_output(filename: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]:
    """
    Parse DOS output file from dos.x.
    
    Returns
    -------
    energy : np.ndarray
    dos : np.ndarray
    idos : np.ndarray (integrated DOS)
    fermi : float (Fermi energy)
    """
    energy = []
    dos = []
    idos = []
    fermi = None
    
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('#'):
                match = re.search(r'EFermi\s*=\s*([\d.+-]+)', line)
                if match:
                    fermi = float(match.group(1))
                continue
            
            parts = line.split()
            if len(parts) >= 2:
                energy.append(float(parts[0]))
                dos.append(float(parts[1]))
                if len(parts) >= 3:
                    idos.append(float(parts[2]))
    
    return np.array(energy), np.array(dos), np.array(idos) if idos else None, fermi

print("DOS calculation functions defined.")
print("\nExample dos.x input:")
print(generate_dos_input('silicon'))

---

## 5. Band Gap Problem in DFT

### Why DFT Underestimates Band Gaps

| Material | DFT (PBE) | Experiment | Error |
|----------|-----------|------------|-------|
| Si | ~0.5 eV | 1.17 eV | -57% |
| Ge | ~0.0 eV | 0.74 eV | -100% |
| GaAs | ~0.5 eV | 1.52 eV | -67% |
| MgO | ~4.5 eV | 7.8 eV | -42% |

### Solutions

1. **GW calculations**: Most accurate, but expensive
2. **Hybrid functionals (HSE06)**: Good balance of accuracy and cost
3. **Scissors operator**: Empirical shift (quick and dirty)
4. **DFT+U**: For d/f electron systems

---

## 6. Effective Mass Calculation

In [ ]:
def calculate_effective_mass(k_distances: np.ndarray, energies: np.ndarray,
                              k_index: int, lattice_constant: float) -> float:
    """
    Calculate effective mass from band curvature.
    
    m* = ℏ² / (d²E/dk²)
    
    Parameters
    ----------
    k_distances : np.ndarray
        K-point distances (in reciprocal space units)
    energies : np.ndarray
        Band energies in eV
    k_index : int
        Index of k-point for mass calculation (usually band extremum)
    lattice_constant : float
        Lattice constant in Angstrom
    
    Returns
    -------
    float : Effective mass in units of electron mass (m0)
    """
    # Need at least 3 points for second derivative
    if k_index < 1 or k_index >= len(k_distances) - 1:
        print("k_index must not be at boundary")
        return None
    
    # Finite difference for second derivative
    dk = k_distances[k_index + 1] - k_distances[k_index]
    d2E_dk2 = (energies[k_index + 1] - 2 * energies[k_index] + energies[k_index - 1]) / (dk ** 2)
    
    # Convert units: E in eV, k in 1/Å
    # m* = ℏ² / (d²E/dk²)
    # With ℏ² in eV·Å²·s²/kg and converting to m0 units
    
    hbar_sq = 7.6199  # ℏ² in eV·Å²·m0
    
    if abs(d2E_dk2) < 1e-10:
        return float('inf')
    
    m_star = hbar_sq / d2E_dk2
    
    return m_star

print("Effective mass calculation function defined.")
print("\nNote: For accurate effective masses, use dense k-mesh near extrema.")

---

## Summary

### Electronic Property Calculations

| Property | Method | Key Parameters |
|----------|--------|----------------|
| Band structure | SCF → bands → bands.x | K-path, nbnd |
| DOS | SCF → NSCF → dos.x | Dense k-mesh, tetrahedra |
| PDOS | ... → projwfc.x | Orbital projections |
| Effective mass | From band curvature | d²E/dk² |

### Key Reminders

1. **ALWAYS verify stability first**
2. **DFT underestimates band gaps** - don't trust quantitatively
3. **Use dense k-mesh for DOS** - at least 2× the SCF grid
4. **Qualitative trends are reliable** - direct vs indirect, orbital character

### Next Notebook
→ **09_Advanced_Properties.ipynb**: Optical, transport, and other properties